# Cornershop's Data Science Test

## Dev: Mateus Broilo
mateus.brilo90@gmail.com

* Objectives: Load Datasets and perform 

 # Summary

* Initials
    * Loading Libraries and Packages
    * Configurations
    * Global Variables
    * Function's Definition
        * Expand dates, create day of the week column and transform time to pure number
* Loading data
    * Creating resources


# Initials
* Loading Libraries and Packages
* Configuration
* Global Variables

## Loading Libraries and Packages 

In [1]:
# general
import warnings
import time
import gc

#data
import pandas as pd
import numpy as np
from datetime import (datetime, date)

## visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ls -l ../data/

total 504
-rw-rw-r-- 1 broilo broilo 514550 mai 20 15:58 20220520_corrected_sheet1.csv


## Configurations

In [3]:
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

rc={'font.size': 14, 'axes.labelsize': 14, 'legend.fontsize': 12, 
    'axes.titlesize': 24, 'xtick.labelsize': 14, 'ytick.labelsize': 12}

sns.set(rc=rc)
sns.set_palette("Set2")
sns.set_style("darkgrid")#sns.set_style("whitegrid")

## Global Variables

In [5]:
PATH_DATA = "../data/"
DATA = "20220520_corrected_sheet1.csv"
DATE = time.strftime("%Y%m%d")

## Function's Definition

### Expand dates, create day of the week column and transform time to pure number

In [92]:
def dateTimeConverter(dataset, col1, col2):
    """
    Describe the function!
    """
    dataset['Data'] = dataset[col1].astype(str) + ' ' + dataset[col2].astype(str)
    day_of_week = list(pd.to_datetime(dataset['Data'], format='%Y-%m-%d').dt.dayofweek)
    dataset['day_of_week'] = day_of_week
    dataset[[col2+'_HH', col2+'_MIN', col2+'_SEC']] = dataset[col2].str.split(":", expand=True)
    dataset['pure_time'] = dataset[col2+'_HH'].astype(int) + (dataset[col2+'_MIN'].astype(int) / 60) + (dataset[col2+'_SEC'].astype(float) / 3600)
    
    del dataset[col2+'_HH'], dataset[col2+'_MIN'], dataset[col2+'_SEC']
        
    return dataset

# Loading Datasets
* Creating resources

In [29]:
df = pd.read_csv(PATH_DATA + DATA, sep=',')
print(df.shape)
df

(11128, 5)


,Dia,Hora,Valor,Cartão,CBK
0,2015-05-01,00:01:54,36.54,536518******2108,Não
1,2015-05-01,00:03:46,36.54,536518******2108,Não
2,2015-05-01,00:08:50,69.00,453211******1239,Não
3,2015-05-01,00:27:00,193.43,548827******1705,Não
4,2015-05-01,01:32:46,132.00,531681******9778,Não
...,...,...,...,...,...
11123,2015-05-30,23:07:01,53.00,514868******7409,Não
11124,2015-05-30,23:08:47,15.00,439354******5281,Não
11125,2015-05-30,23:15:24,20.00,549167******1648,Não
11126,2015-05-30,23:17:41,70.00,518759******8384,Não


## Creating resources

In [46]:
df['Data'] = df.Dia.astype(str) + ' ' + df.Hora.astype(str)
len(df.Data[0])

19

In [55]:
df[df.Data == df.Data[0]].index

Int64Index([0], dtype='int64')

In [79]:
idx = []
for i in range(len(df.Data)):
    if len(df.Data[i]) > 19 or len(df.Data[i]) < 19:
        idx.append(df[df.Data == df.Data[i]].index.values[0])
    else:
        None
idx

[7779]

In [80]:
df.drop(index=idx, axis=0, inplace=True)

In [96]:
    col1, col2 = 'Dia', 'Hora'
    day_of_week = list(pd.to_datetime(df[col1], format='%Y-%m-%d').dt.day_name())#.dayofweek)
    df['day_of_week'] = day_of_week
    df[[col1+'_Ano', col1+'_Mês', col1+'_Dia']] = df[col1].str.split("-", expand=True)
    df[[col2+'_HH', col2+'_MIN', col2+'_SEC']] = df[col2].str.split(":", expand=True)
    df['pure_time'] = df[col2+'_HH'].astype(int) + (df[col2+'_MIN'].astype(int) / 60) + (df[col2+'_SEC'].astype(float) / 3600)
    
    del df[col1+'_Ano'], df[col1+'_Mês'], df[col2+'_HH'], df[col2+'_MIN'], df[col2+'_SEC']

In [97]:
df

,Dia,Hora,Valor,Cartão,CBK,Data,day_of_week,pure_time,Dia_Dia
0,2015-05-01,00:01:54,36.54,536518******2108,Não,2015-05-01 00:01:54,Friday,0.031667,01
1,2015-05-01,00:03:46,36.54,536518******2108,Não,2015-05-01 00:03:46,Friday,0.062778,01
2,2015-05-01,00:08:50,69.00,453211******1239,Não,2015-05-01 00:08:50,Friday,0.147222,01
3,2015-05-01,00:27:00,193.43,548827******1705,Não,2015-05-01 00:27:00,Friday,0.450000,01
4,2015-05-01,01:32:46,132.00,531681******9778,Não,2015-05-01 01:32:46,Friday,1.546111,01
...,...,...,...,...,...,...,...,...,...
11123,2015-05-30,23:07:01,53.00,514868******7409,Não,2015-05-30 23:07:01,Saturday,23.116944,30
11124,2015-05-30,23:08:47,15.00,439354******5281,Não,2015-05-30 23:08:47,Saturday,23.146389,30
11125,2015-05-30,23:15:24,20.00,549167******1648,Não,2015-05-30 23:15:24,Saturday,23.256667,30
11126,2015-05-30,23:17:41,70.00,518759******8384,Não,2015-05-30 23:17:41,Saturday,23.294722,30


In [93]:
### Expand dates, create day of the week column and transform time to pure number
dateTimeConverter(df, 'Dia', 'Hora')

print(df.shape)
df.head()

(11127, 8)


,Dia,Hora,Valor,Cartão,CBK,Data,day_of_week,pure_time
0,2015-05-01,00:01:54,36.54,536518******2108,Não,2015-05-01 00:01:54,4,0.031667
1,2015-05-01,00:03:46,36.54,536518******2108,Não,2015-05-01 00:03:46,4,0.062778
2,2015-05-01,00:08:50,69.00,453211******1239,Não,2015-05-01 00:08:50,4,0.147222
3,2015-05-01,00:27:00,193.43,548827******1705,Não,2015-05-01 00:27:00,4,0.450000
4,2015-05-01,01:32:46,132.00,531681******9778,Não,2015-05-01 01:32:46,4,1.546111
